# Imports & Definitions

In [1]:
from SVD_utils import *
from Error_utils import *
import json
import numpy as np

# Creating arrayD

In [5]:
deleteArrays(D=True)
createDB("arrayD", "Temperatures", (num_images, image_height, image_width), (tile_thickness, tile_height, tile_width))
max_val, min_val, data_avg = populateArrayD()
vRange = max_val - min_val

In [6]:
param_dict = {}
parameter_names = ["max_val", "min_val", "data_avg", "vRange"]

for i in parameter_names:
    
    param_dict[i] = float(globals()[i])


with open("parameters.json", "w") as f:
    json.dump(param_dict, f)

In [2]:
with open("parameters.json", "r") as f:
    param_dict = json.load(f)

for key, value in param_dict.items():
    globals()[key] = np.float32(value)

# SVD related work

In [3]:
deleteArrays(D=False)
createDB("arrayUr", "U", db_shape=(num_SVD_matrices, SVD_shape[0], k), tile_shape=(1,SVD_shape[0],k), data_type=np.float32)
createDB("arraySr", "S", db_shape=(num_SVD_matrices, k, 1), tile_shape=(1,k,1), data_type=np.float32)
createDB("arrayVr", "V", db_shape=(num_SVD_matrices, k, SVD_shape[1]), tile_shape=(1,k,SVD_shape[1]), data_type=np.float32)

In [4]:
max_error, min_error = compressData(data_avg)

100%|██████████| 1215/1215 [03:55<00:00,  5.17it/s]


In [5]:
# check quantizability
step = (max_error - min_error) / 256
do_quant = (step/2) / vRange < eps
print(do_quant)

True


In [6]:
E_datatype = np.uint8 if do_quant else np.float32
createDB("arrayE", "E", (num_SVD_matrices, SVD_shape[0], SVD_shape[1]), (1, SVD_shape[0], SVD_shape[1]), sparse=True, data_type=E_datatype)
populateErrorArray(max_error, min_error, data_avg, do_quant)

100%|██████████| 1215/1215 [08:14<00:00,  2.46it/s]


In [7]:
createDB("arrayD_prime", "Temperatures", (num_images, image_height, image_width), (tile_thickness, tile_height, tile_width))

reconstructAndCheck(max_error, min_error, data_avg, do_quant)

100%|██████████| 1215/1215 [04:46<00:00,  4.24it/s]

Check successful


In [10]:
getCompressionRatio()

16624609629
489994108 982273717
Compression ratio p: 11.291837902522932


11.291837902522932

In [ ]:
11.29